# 🗂️ Topic Modelling

Traditional topic modeling techniques like LDA (Latent Dirichlet Allocation) use word frequency patterns to discover topics – essentially clusters of words that frequently co-occur in documents. While effective, LDA has limitations (it treats words as unrelated entities, so “car” and “automobile” wouldn’t be linked as same topic without explicit signals).

Neural topic modeling incorporates embeddings to improve topic coherence. BERTopic is one such modern method that has gained popularity. It leverages transformer-based embeddings and a class-based TF-IDF procedure to create clusters of documents and extract topics.

BERTopic uses a neural network model to create embeddings of the text, which are then clustered to form topics. It's different stages are highly configurable, but the basic flow is as follows:
1. **Embedding**: Convert text documents into vector representations using a pre-trained model (like BERT).
2. **Dimensionality Reduction**: Reduce the high-dimensional embeddings to a lower-dimensional space using techniques like UMAP.
3. **Clustering**: Group the reduced embeddings into clusters (i.e., topics) based on their similarity using algorithms like HDBSCAN.
4. **Tokenization**: Tokenize the clusters to prepare for topic representation.
5. **Weighting**: Calculate the importance of words in each cluster using class-based TF-IDF (c-TF-IDF) or similar methods.
6. **Topic Representation**: Provide a representation (and/or label) of each topic, based on the most important words in each cluster and, optionally, example documents.

The result is a set of topics, each described by a set of top keywords, and every document is assigned a *single* topic (unlike LDA where each document is considered a mixture of topics).

Let's use BERTopic to identify topics in a dataset of articles from the Guardian newspaper.

In [ ]:
import pandas as pd

df = pd.read_csv('data/guardian_articles.csv')

df

## 🔤 Simple known topics

Since BERTopic uses a neural network model, we do not need to preprocess the text as much as with traditional methods.

We do choose to remove stopwords at the tokenisation/weighting c-TF-IDF stage (the lines on `vectorizer_model`), but this is done after the embedding and clustering steps, so only affects the display of the final topic representative words, not the clustering itself.

We start off with a simple task taking a sample of "Opinion" and "Football" articles. When we run BERTopic, we expect it will be able to identify articles as either Opinion/Football, based on solely the text content. This is an entirely unsupervised task (though BERTopic can also be customised to run in a semi-supervised way).

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

# Filter the DataFrame for specific sections and drop rows with NaN in 'bodyContent'
sample_df = df[df['sectionName'].isin(['Opinion', 'Football'])
               ].dropna(subset='bodyContent').sample(n=1000, random_state=42) # Sample 1000 rows for speed today
docs = sample_df['bodyContent'].tolist()

vectorizer_model = CountVectorizer(stop_words="english") # only used for c-TF-IDF stage
topic_model = BERTopic(vectorizer_model=vectorizer_model) #  create BERTopic model
topics, probs = topic_model.fit_transform(docs) # fit the model to the documents
topic_info = topic_model.get_topic_info() # get topic information
topic_info # display the topics found

Let's take a look at the representative words for each topic in more detail

In [ ]:
for topic_id in range(min(topic_info['Topic']), max(topic_info['Topic'] + 1)):
    print(f"Topic {topic_id}: ")
    display(pd.Series(dict(topic_model.get_topic(topic_id)), name='c-TF-IDF'))

This looks pretty promising - one topic covers words associated with politics / current affairs, the other with football. Let's compare against the actual labels

In [ ]:
# compare topic assignment against df labels
sample_df['topic'] = topics
display(sample_df[['sectionName', 'topic']].value_counts().unstack().fillna(0).astype(int))

Very good, but not perfect agreement, which is to be expected given the unsupervised nature of the task. The model has identified two distinct topics, but there are some articles that don't fit neatly into either category. We see these misclassifications below as "Football" stories that deal with serious peolitical issues, and "Opinion" stories that are about football / sport.

In [ ]:
#  display the misclassified documents
print('Football articles misclassified as Opinion:')
for t in sample_df[(sample_df['sectionName'] == 'Football') & (sample_df['topic'] == 0)]['webTitle']:
    print(t)

print()

print('Opinion articles misclassified as Football:')
for t in sample_df[(sample_df['sectionName'] == 'Opinion') & (sample_df['topic'] == 1)]['webTitle']:
    print(t)


### 🏋️ Exercise

1. Remove the `vectorizer_model=vectorizer_model` argument from the `BERTopic` constructor, and re-run the model. What happens to the topics?
2. Try selecting some articles from different sections of the Guardian, such "Politics", "World news", "US news". What do we find for topics here?

## 🔍 Unknown Topics

The last task was actually pretty simple, and not actually that useful, but served as an illustration of how BERTopic works and and indication of its performance. Let's now try a more challenging (and more suitable) task, where we don't have any labels to compare against. We want to use it to *discover* the underlying topics in a set of articles, without any prior knowledge of what those topics might be. We will focus just on Opinion articles, which will contain Opinion pieces on a wide range of topics, from politics to culture to sport. We want to ask what are the main Opinions topics being covered.

In [ ]:
# Filter the DataFrame for Opinion articles and drop rows with NaN in 'bodyContent'
sample_df = df[df['sectionName'] == 'Opinion'
               ].dropna(subset='bodyContent').sample(n=1000, random_state=42) # Sample 1000 rows for speed today
docs = sample_df['bodyContent'].tolist()

vectorizer_model = CountVectorizer(stop_words="english") # only used for c-TF-IDF stage
topic_model = BERTopic(vectorizer_model=vectorizer_model) #  create BERTopic model
topics, probs = topic_model.fit_transform(docs) # fit the model to the documents
topic_info = topic_model.get_topic_info() # get topic information
topic_info # display the topics found

We see a wider variety of interesting topics here! They seem pretty coherent on first glance, but let's take a closer look at the representative words for each topic to see if they make sense.

Firstly, topic -1 is a catch-all for articles that don't fit well into any of the other topics. This is common in topic modeling, as it allows the model to handle outliers or noise in the data.

The remaining topics cover a range of suhjects, such as Brexit, Trump, Covid, etc.

In [ ]:
for topic_id in range(min(topic_info['Topic']), max(topic_info['Topic'] + 1)):
    print(f"Topic {topic_id}: {topic_info.loc[topic_info['Topic'] == topic_id, 'Name'].values[0]}")
    display(pd.Series(dict(topic_model.get_topic(topic_id)), name='c-TF-IDF'))

## 📊 Visualizing Topics

We can visualize the topics using the `visualize_topics` method, which will show us a plot of the topics and their relationships. This can help us understand how the topics are related to each other and identify any potential overlaps or clusters. Each axis represents the first two components of the UMAP dimensionality reduction, and each point represents a topic. The size of the point indicates the number of articles in that topic.

In [ ]:
topic_model.visualize_topics()

We can also investigate how these have changed over time:

In [ ]:
timestamps = pd.to_datetime(sample_df['webPublicationDate']).tolist()
topics_over_time = topic_model.topics_over_time(docs, timestamps, nr_bins=50)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)

One can adust various parameters to control the number of topics, or manually merge topics that are similar. BERTopic also supports hierarchical topic modeling, which allows you to create a tree structure of topics and subtopics. This can be useful for further exploring the relationships between topics and understanding how they are related to each other.

### 🏋️ Exercise

1. Explore the [BERTopic documentation](https://maartengr.github.io/BERTopic/getting_started/quickstart/quickstart.html). Tweak some of the arguments / parameters in our analysis of news topics to explore how they affect the topics, or run additional stages of analysis. For example, you could:
   - Change the `min_topic_size` parameter to see how it affects the number of topics.
   - Explore [hierarchical topic modeling](https://maartengr.github.io/BERTopic/getting_started/hierarchicaltopics/hierarchicaltopics.html).
   - Explore using a different [embedding model](https://maartengr.github.io/BERTopic/getting_started/embeddings/embeddings.html) (default is all-MiniLM-L6-v2) or [representation model](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html) (default is c-TF-IDF).
  

## 🌯 Wrap up

- Topic modelling helps uncover latent themes in documents
- BERTopic uses embeddings and clustering for flexible, modern topic discovery
- Topics are **approximations**, not “truths”
- Preprocessing and hyperparameters change what topics emerge
- Topic models are **exploratory** tools — great for surfacing patterns, not precise classification
